<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Лемматизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    RandomizedSearchCV,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import re
import spacy

RANDOM_STATE = 43
TEST_SIZE = 0.25

In [2]:
df = pd.read_csv(
    'https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=0
)

In [3]:
df['text'].duplicated().sum()

0

In [4]:
df['toxic'].value_counts() / len(df)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Присутствует сильный дисбаланс классов. Токсичных комментариев всего 10%

### Лемматизация

In [5]:
def lemmatize_it(text, nlp_object):
    '''
    Лематизирует текст, удаляет стоп-символы и лишние пробелы
    text - текст, который надо лемматизировать
    nlp_object - объект, проводящий лемматизацию
    '''
    doc = nlp(text)
    lemm = " ".join([token.lemma_ for token in doc])
    clear = re.sub(r'[^a-zA-Z ]', ' ', lemm.lower())
    return ' '.join(clear.split())

In [6]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [7]:
df['lemm_text'] = df['text'].apply(lambda x: lemmatize_it(x, nlp))

In [8]:
df.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i be see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i be really not try to edit war it be ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can not make any real suggestion on imp...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


In [9]:
df['lemm_text'].duplicated().sum()

1327

После лемматизации обнаружились дубликаты. Уберем их и проверим, изменился ли баланс классов.

In [10]:
df = df.drop_duplicates(subset=['lemm_text'])
df['toxic'].value_counts() / len(df)

0    0.898433
1    0.101567
Name: toxic, dtype: float64

Баланс классов не изменился

## Обучение

In [11]:
X = df['lemm_text']
y = df['toxic']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

In [13]:
count_tf_idf = TfidfVectorizer(stop_words='english')

In [15]:
pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("model", LogisticRegression(random_state=RANDOM_STATE)),
    ]
)
pipeline

Pipeline(steps=[('vect', TfidfVectorizer()),
                ('model', LogisticRegression(random_state=43))])

In [16]:
# LogReg
logreg_params = {}

logreg_params['vect__max_df'] = [0.2, 0.4, 0.6, 0.8, 1.0]
logreg_params['vect__min_df'] = [1, 3, 5, 10]
logreg_params['vect__norm'] = ['l1', 'l2']

logreg_params['model__l1_ratio'] = [0.6, 0.7, 0.8]
logreg_params['model__penalty'] = ['elasticnet']
logreg_params['model__solver'] = ['saga']
logreg_params['model'] = [LogisticRegression(random_state=RANDOM_STATE)]

# Forest
forest_params = {}

forest_params['vect__max_df'] = [0.2, 0.4, 0.6, 0.8, 1.0]
forest_params['vect__min_df'] = [1, 3, 5, 10]
forest_params['vect__norm'] = ['l1', 'l2']

forest_params['model__max_depth'] = [2, 5, 10]
forest_params['model__min_samples_split'] = [10, 50, 100, 250, 500, 1000]
forest_params['model'] = [RandomForestClassifier(random_state=RANDOM_STATE)]

# SVC
svc_params = {}

svc_params['vect__max_df'] = [0.2, 0.4, 0.6, 0.8, 1.0]
svc_params['vect__min_df'] = [1, 3, 5, 10]
svc_params['vect__norm'] = ['l1', 'l2']

svc_params['model__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
svc_params['model'] = [SVC()]

params = [logreg_params, forest_params, svc_params]

In [19]:
rnd = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=params,
    n_iter=10,
    cv=5,
    n_jobs=-1,
    scoring='f1',
    error_score='raise',
)

rnd.fit(X_train, y_train)

RandomizedSearchCV(cv=5, error_score='raise',
                   estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                             ('model',
                                              LogisticRegression(random_state=43))]),
                   n_jobs=-1,
                   param_distributions=[{'model': [LogisticRegression(random_state=43)],
                                         'model__l1_ratio': [0.6, 0.7, 0.8],
                                         'model__penalty': ['elasticnet'],
                                         'model__solver': ['saga'],
                                         'vect__max_df': [0.2, 0.4, 0.6, 0.8,
                                                          1.0],
                                         '...
                                        {'model': [RandomForestClassifier(random_state=43)],
                                         'model__max_depth': [2, 5, 10],
                                         'model__min_samples_split': [10, 50,
                                                                      100, 250,
                                                                      500,
                                                                      1000],
                                         'vect__max_df': [0.2, 0.4, 0.6, 0.8,
                                                          1.0],
                                         'vect__min_df': [1, 3, 5, 10],
                                         'vect__norm': ['l1', 'l2']},
                                        {'model': [SVC()],
                                         'model__C': [0.01, 0.1, 1, 10, 100],
                                         'vect__max_df': [0.2, 0.4, 0.6, 0.8,
                                                          1.0],
                                         'vect__min_df': [1, 3, 5, 10],
                                         'vect__norm': ['l1', 'l2']}],
                   scoring='f1')

In [21]:
results_df = pd.DataFrame(rnd.cv_results_)
results_df = results_df.sort_values(by=["rank_test_score"])
results_df = results_df.set_index(
    results_df["params"].apply(
        lambda x: "_".join(str(val) for val in x.values())
    )
).rename_axis("kernel")
results_df['mean_test_score'] *= -1
results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]

,params,rank_test_score,mean_test_score,std_test_score
kernel,,,,
l2_1_0.6_saga_elasticnet_0.8_LogisticRegression(random_state=43),"{'vect__norm': 'l2', 'vect__min_df': 1, 'vect_...",1,-0.766361,0.002817
l2_5_0.8_saga_elasticnet_0.7_LogisticRegression(random_state=43),"{'vect__norm': 'l2', 'vect__min_df': 5, 'vect_...",2,-0.764411,0.003854
l1_10_0.4_10_SVC(),"{'vect__norm': 'l1', 'vect__min_df': 10, 'vect...",3,-0.737216,0.004292
l1_5_1.0_saga_elasticnet_0.7_LogisticRegression(random_state=43),"{'vect__norm': 'l1', 'vect__min_df': 5, 'vect_...",4,-0.576133,0.010505
l2_10_1.0_500_10_RandomForestClassifier(random_state=43),"{'vect__norm': 'l2', 'vect__min_df': 10, 'vect...",5,-0.003645,0.003204
l1_1_0.6_500_5_RandomForestClassifier(random_state=43),"{'vect__norm': 'l1', 'vect__min_df': 1, 'vect_...",6,-0.000000,0.000000
l1_10_0.6_1000_5_RandomForestClassifier(random_state=43),"{'vect__norm': 'l1', 'vect__min_df': 10, 'vect...",6,-0.000000,0.000000
l1_5_1.0_100_2_RandomForestClassifier(random_state=43),"{'vect__norm': 'l1', 'vect__min_df': 5, 'vect_...",6,-0.000000,0.000000
l1_3_0.6_50_5_RandomForestClassifier(random_state=43),"{'vect__norm': 'l1', 'vect__min_df': 3, 'vect_...",6,-0.000000,0.000000


Даже с помощью randomized search удалось найти удовлетворяющую нас модель всего лишь за 10 итераций.

In [27]:
best_estimator = rnd.best_estimator_

vect = best_estimator[0]
model = best_estimator[1]

In [29]:
tf_idf_test = vect.transform(X_test)

In [30]:
y_pred = model.predict(tf_idf_test)
f1_score(y_test, y_pred)

0.7733524355300859

## Выводы

**Перед нами стояла задача:**

- Разработать модель, которая бы маркировала токсичные комментарии в интернет-магазине;

**Что мы сделали:**

- Загрузили и обработали данные: провели лемматизацию, избавились от дублей, возникших после неё;
- Вычислили TF IDF для лемматизированных строк;
- Проверили несколько моделей логистической регрессии, опорных векторов и случайного леса. Подобрали необходимые гиперпараметры для модели и для векторизатора;

**Результат:**
- Логистическая регрессия справилась с заданием, обеспечив F1 = 0.77, что нам и требовалось;
- На результате положительно сказалась регуляризация elasticnet с долей L1 = 0.8